In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [4]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.]
Size: 16


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [5]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [6]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:259]
    features_test_csv  = features_csv_files[259:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_test_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_test_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 259
Test matches: 39
Train-set shape: (1040694, 228)
Test-set shape: (164281, 228)


In [7]:
    # shuffling data-sets agin
    df_train = df_train[df_train['pitch_index']!=0]
    df_test = df_test[df_test['pitch_index']!=0]
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [8]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (317895, 228)

Pitch index counts
------------------
1	: 7402
2	: 36809
3	: 7301
4	: 20896
5	: 25932
6	: 20703
7	: 30051
8	: 31265
9	: 31016
10	: 23895
11	: 19192
12	: 25869
13	: 12158
14	: 11210
15	: 14196


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,76.37500,19.281250,0.799805,1.0,1.0,77.12500,23.609375,67.687500,24.953125,81.31250,...,8.843750,1.889648,0.00000,0.000000,61.062500,8.843750,0.0000,0.00000,0.000000,10.0
1,50.37500,14.992188,1.049805,-1.0,1.0,43.03125,15.273438,32.812500,17.312500,57.65625,...,31.265625,0.250000,33.34375,31.265625,0.000000,0.000000,0.0000,0.00000,0.000000,4.0
2,74.12500,33.218750,0.899902,-1.0,-1.0,76.62500,36.218750,77.562500,27.703125,85.56250,...,38.750000,1.040039,56.68750,38.750000,0.000000,0.000000,0.0000,0.00000,0.000000,12.0
3,20.21875,36.031250,1.240234,1.0,1.0,8.25000,29.875000,9.171875,32.843750,20.65625,...,18.953125,2.070312,0.00000,0.000000,7.570312,18.953125,0.0000,0.00000,0.000000,2.0
4,72.50000,44.781250,1.650391,1.0,-1.0,81.18750,53.156250,69.812500,50.781250,83.06250,...,57.218750,1.900391,63.62500,53.750000,0.000000,0.000000,79.9375,60.65625,3.169922,12.0


### Test Set

In [10]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (50323, 228)

Pitch index counts
------------------
1	: 1228
2	: 6088
3	: 1192
4	: 3284
5	: 4051
6	: 3075
7	: 5125
8	: 4811
9	: 4986
10	: 4028
11	: 2715
12	: 4026
13	: 2024
14	: 1506
15	: 2184


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,70.000000,29.9375,1.589844,1.0,1.0,67.687500,37.656250,61.593750,37.56250,77.37500,...,51.03125,3.779297,0.00000,0.00000,0.000000,0.0000,57.25000,51.03125,3.779297,8.0
1,21.265625,29.7500,1.660156,-1.0,-1.0,13.359375,16.953125,12.226562,18.68750,45.15625,...,0.00000,0.000000,0.00000,0.00000,0.000000,0.0000,0.00000,0.00000,0.000000,2.0
2,25.609375,50.0625,3.300781,-1.0,1.0,18.015625,48.312500,17.187500,49.25000,55.37500,...,43.78125,4.191406,0.00000,0.00000,0.000000,0.0000,16.71875,43.78125,4.191406,3.0
3,39.781250,51.0625,0.970215,1.0,-1.0,32.281250,51.156250,32.906250,48.96875,57.59375,...,63.06250,2.970703,0.00000,0.00000,27.484375,63.0625,0.00000,0.00000,0.000000,6.0
4,64.125000,37.0625,0.990234,-1.0,-1.0,57.375000,47.281250,52.187500,47.28125,74.06250,...,29.09375,2.210938,54.65625,19.09375,0.000000,0.0000,45.09375,39.09375,3.580078,9.0


## Exporting

In [11]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}.csv'.format(dataset='test'), index=False)

In [12]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,50.37500,14.992188,1.049805,-1.0,1.0,43.03125,15.273438,32.812500,17.312500,57.65625,...,31.265625,0.250000,33.34375,31.265625,0.000000,0.000000,0.00000,0.000000,0.000000,4.0
2,74.12500,33.218750,0.899902,-1.0,-1.0,76.62500,36.218750,77.562500,27.703125,85.56250,...,38.750000,1.040039,56.68750,38.750000,0.000000,0.000000,0.00000,0.000000,0.000000,12.0
3,20.21875,36.031250,1.240234,1.0,1.0,8.25000,29.875000,9.171875,32.843750,20.65625,...,18.953125,2.070312,0.00000,0.000000,7.570312,18.953125,0.00000,0.000000,0.000000,2.0
4,72.50000,44.781250,1.650391,1.0,-1.0,81.18750,53.156250,69.812500,50.781250,83.06250,...,57.218750,1.900391,63.62500,53.750000,0.000000,0.000000,79.93750,60.656250,3.169922,12.0
5,55.21875,27.406250,3.070312,1.0,1.0,43.81250,32.906250,41.281250,14.492188,65.68750,...,29.312500,5.816406,0.00000,0.000000,0.000000,0.000000,39.71875,36.875000,6.878906,6.0
6,59.87500,37.593750,4.691406,1.0,-1.0,53.65625,32.031250,52.500000,37.437500,71.50000,...,37.906250,3.160156,0.00000,0.000000,0.000000,0.000000,35.71875,37.906250,3.160156,9.0
7,37.56250,33.218750,4.679688,1.0,1.0,20.56250,38.250000,20.750000,34.562500,50.56250,...,34.750000,1.580078,0.00000,0.000000,25.062500,34.750000,0.00000,0.000000,0.000000,6.0
8,51.93750,47.843750,2.119141,1.0,-1.0,53.43750,52.125000,37.593750,65.187500,62.37500,...,45.718750,3.285156,0.00000,0.000000,30.953125,36.750000,39.25000,54.687500,4.050781,6.0
9,77.75000,15.671875,0.669922,1.0,-1.0,89.37500,24.187500,71.812500,28.406250,90.06250,...,28.562500,2.839844,0.00000,0.000000,67.312500,28.875000,55.09375,28.234375,3.480469,13.0
